1) singer 테이블 만들기
- id (Auto increment 필드)
- name (가수 이름)
- follower(팔로우 수)  -> 인증 표시 있을경우만 가져오게 만들기

2) song 테이블 만들기  -> singer_id를 만들때 FK 누르면 Referenced tabld에서 singer 선택하면 된다.
- id (Auto increment 필드)
- singer_id(FK)
- title(제목)
- album(앨범)

3) 멜론 TOP100 을 수집
- 멜론 TOP100에 존재하는 가수들을 singer 테이블에 저장 
                   -> 이게 어려울것. 중복된 가수가 있을 수 있음. 따라서 기존에 있으면 추가하면 안된다.
- 각 가수의 곡들은 song 테이블에 저장
- 단, singer 테이블에서 follower는 0으로 저장

4) 인스타그램에서 가수명으로 검색한 후 인증마크 달려 있는 계정의 follower 가져와서 데이터베이스의 follower 값 업데이트

5) Hint!!
    <가수 저장>
    1. 멜론 TOP100에서 가수를 읽어옴
    2. singer 테이블에 가수명 저장 =>insert 구문
    
    <노래 저장>
    1. 멜론 TOP100에서 제목, 앨범, 가수명을 읽어옴
    2. 노래 저장시 필요정보: 제목, 앨범, 가수_id
    3. DB에서 가수명으로 가수_id를 구함 => Select 구문
    4. song테이블에 노래 저장 -> insert 구문
    
    <가수 팔로우 갱신>
    1. singer 테이블에서 가수 전체를 읽어옴 => cursor.fetchall()
    2. 가수 전체 정보 : 가수_id, 가수명
    3. 인스타그램 접속해서 가수명으로 검색
    4. 인증마크 있는 지 확인, 있으면 클릭
    5. 인스타그램 계정 페이지에서 팔로워 수 가져옴
    6. singer 테이블에서 갱신을 하는데 가수_id를 활용해서 follower 업데이트
   

In [9]:
# DB 입력 파트

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import traceback # 에러 발생했을 때 
import pymysql

options = webdriver.ChromeOptions()
options.add_argument("--user-data-dir=C:\\Users\\user\\Desktop\\MyChrome")

db = pymysql.connect(host="localhost", port=3306, user="root", password = "1234", db = "sba")
cursor = db.cursor() 

service = Service()
d = webdriver.Chrome(service=service, options=options)

already = []

try:
    d.get("https://www.melon.com/chart/index.htm?dayTime=2023080314")
    
    area = d.find_element(By.CSS_SELECTOR, ".service_list_song")
    trs = area.find_elements(By.CSS_SELECTOR, "table>tbody>tr")

    for tr in trs:

        singer = tr.find_element(By.CSS_SELECTOR,".ellipsis.rank02 a")
        singer = singer.text.replace("'", "")

        print(singer)
        if singer not in already:
            sql = f""" 
            INSERT INTO singer 
            VALUES
                (NULL, '{singer}',"0");     

            """
            cursor.execute(sql)
            db.commit()
            already.append(singer)
            
            
            
        title = tr.find_element(By.CSS_SELECTOR,".ellipsis.rank01 a")
        album = tr.find_element(By.CSS_SELECTOR,".ellipsis.rank03 a")
        title = title.text.replace("'", "")
        album = album.text.replace("'", "")
        
        
        print(title)
        sql = f"""
            SELECT id FROM singer WHERE name='{singer}'
        """
        cursor.execute(sql)
        a = cursor.fetchone()
        a_num = a[0]
        print(a_num)
        
        sql = f""" 
        INSERT INTO song 
        VALUES
            (NULL, {a_num}, '{title}','{album}');

        """
        cursor.execute(sql)
            

    db.commit()
    db.close()
    
except Exception as e:            # except는 필수! 오류났을 경우 error 메시지 출력
    traceback.print_exc()
finally:                          # finally는 선택사항 error가 날지도 모르는 사항
    time.sleep(2)
    d.close()
    d.quit()
      

NewJeans
Super Shy
101
정국
Seven (feat. Latto) - Clean Ver.
102
NewJeans
ETA
101
(여자)아이들
퀸카 (Queencard)
103
박재정
헤어지자 말해요
105
IVE (아이브)
I AM
104
LE SSERAFIM (르세라핌)
이브, 프시케 그리고 푸른 수염의 아내
106
aespa
Spicy
107
Lauv
Steal The Show (From “엘리멘탈”)
108
NewJeans
New Jeans
101
임영웅
사랑은 늘 도망가
109
NewJeans
Hype boy
101
IVE (아이브)
Kitsch
104
임영웅
모래 알갱이
109
LE SSERAFIM (르세라핌)
UNFORGIVEN (feat. Nile Rodgers)
106
임영웅
우리들의 블루스
109
NewJeans
Ditto
101
임영웅
다시 만날 수 있을까
109
임영웅
무지개
109
NewJeans
Cool With You
101
임영웅
이제 나만 믿어요
109
임영웅
London Boy
109
임영웅
Polaroid
109
임영웅
아버지
109
NewJeans
Attention
101
세븐틴 (SEVENTEEN)
손오공
111
임영웅
인생찬가
109
임영웅
A bientot
109
NCT DREAM
Candy
110
NewJeans
OMG
101
임영웅
손이 참 곱던 그대
109
임영웅
사랑해 진짜
109
NCT DREAM
ISTJ
110
임영웅
연애편지
109
지수 (JISOO)
꽃
112
IVE (아이브)
After LIKE
104
윤하 (YOUNHA)
사건의 지평선
114
방탄소년단
Dynamite
113
정국
Still With You
102
임영웅
보금자리
109
IVE (아이브)
LOVE DIVE
104
Charlie Puth
I Dont Think That I Like Her
115
Paul Blanco
Summer (Feat. BE’O (비오))
121
방탄소년단
Butter
113
부석순 (SEVENTEEN

In [21]:
# instagram 파트

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import traceback # 에러 발생했을 때 
import pymysql

options = webdriver.ChromeOptions()
options.add_argument("--user-data-dir=C:\\Users\\user\\Desktop\\MyChrome")

db = pymysql.connect(host="localhost", port=3306, user="root", password = "1234", db = "sba")
cursor = db.cursor() 

service = Service()
d = webdriver.Chrome(service=service, options=options)

cursor.execute("SELECT * FROM singer")
rows = cursor.fetchall()
try:   
    for row in rows:
       
        # Instagram 검색 시작
        d.get("https://www.instagram.com/")    
        time.sleep(5)

        buttons = d.find_elements(By.CSS_SELECTOR, ".xvy4d1p")
        search_button = buttons[2]

        ac = ActionChains(d)
        ac.move_to_element(search_button)
        ac.click()
        ac.pause(1)
        ac.perform()

        elem = d.find_element(By.CSS_SELECTOR, ".x7xwk5j")
        ac = ActionChains(d)
        ac.move_to_element(elem)
        ac.click()

        ac.send_keys(f'{row[1]}')
        ac.send_keys(Keys.RETURN)
        ac.pause(2)
        
        
        ac.move_by_offset(0, 100)
        
        ac.click()
        ac.perform()
        time.sleep(5)
        
        follow = d.find_element(By.CSS_SELECTOR, "._a6hd > ._ac2a").text.replace("'", "")
        print(f"Instagram 팔로우 수: {follow}")
    
            
            
    
        
except Exception as e:            # except는 필수! 오류났을 경우 error 메시지 출력
    traceback.print_exc()
finally:                          # finally는 선택사항 error가 날지도 모르는 사항
    time.sleep(2)
    d.close()
    d.quit()
      

Instagram 팔로우 수: 878.3만
Instagram 팔로우 수: 16.7만
Instagram 팔로우 수: 1118만
Instagram 팔로우 수: 394.1만
Instagram 팔로우 수: 12.9만
Instagram 팔로우 수: 46
Instagram 팔로우 수: 1109만
Instagram 팔로우 수: 192.8만
Instagram 팔로우 수: 38.6만
Instagram 팔로우 수: 1284만
Instagram 팔로우 수: 17.5만
Instagram 팔로우 수: 7478만
Instagram 팔로우 수: 89.7만
Instagram 팔로우 수: 30.2만
Instagram 팔로우 수: 1763만
Instagram 팔로우 수: 65
Instagram 팔로우 수: 192.9만
Instagram 팔로우 수: 84.7만
Instagram 팔로우 수: 2.2만
Instagram 팔로우 수: 4712만
Instagram 팔로우 수: 11.2만
Instagram 팔로우 수: 500
Instagram 팔로우 수: 8.6만
Instagram 팔로우 수: 14.5만


Traceback (most recent call last):
  File "<ipython-input-21-050e44fe1c96>", line 31, in <module>
    buttons = d.find_elements(By.CSS_SELECTOR, ".xvy4d1p")
  File "c:\python37\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 770, in find_elements
    return self.execute(Command.FIND_ELEMENTS, {"using": by, "value": value})["value"] or []
  File "c:\python37\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 346, in execute
    self.error_handler.check_response(response)
  File "c:\python37\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 245, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.111)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A1A813+48355]
	(No symbol) [0x009AC4B1]
	(No symbol) [0x008B5358]
	(No symbol) [0x0089D293]
	(No symb

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=115.0.5790.111)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A1A813+48355]
	(No symbol) [0x009AC4B1]
	(No symbol) [0x008B5358]
	(No symbol) [0x008A70EB]
	(No symbol) [0x008A77AA]
	(No symbol) [0x008A7748]
	(No symbol) [0x0089C2AA]
	(No symbol) [0x0089CB1D]
	(No symbol) [0x009027B8]
	(No symbol) [0x008FA703]
	(No symbol) [0x008D82DC]
	(No symbol) [0x008D93DD]
	GetHandleVerifier [0x00C7AABD+2539405]
	GetHandleVerifier [0x00CBA78F+2800735]
	GetHandleVerifier [0x00CB456C+2775612]
	GetHandleVerifier [0x00AA51E0+616112]
	(No symbol) [0x009B5F8C]
	(No symbol) [0x009B2328]
	(No symbol) [0x009B240B]
	(No symbol) [0x009A4FF7]
	BaseThreadInitThunk [0x763FFEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77477BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77477B8E+238]


In [ ]:
# 인증 hint

tmp = "x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1gslohp x1i64zmx x1n2onr6 x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1"
tmp = tmp.split(" ")
tmp = "." + ".".join(tmp)

marks = d.find_elements(By.CSS_SELECTOR, tmp)
if marks:
    ac.click(marks[0])